In [2]:
# Import necessary packages
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import csv as csv


In [3]:
# Read the train and test data into pandas data framesfrom StringIO import StringIO
from StringIO import StringIO
import requests
import json

# This function includes credentials to your Object Storage.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_f2ced6cdb4894c42945223d461f16d59(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'admin_d62aa33975e0d99158757aee850892d2cdfcf245','domain': {'id': 'b00f5856f86c411e99a1ed33654a0ec7'},
            'password': 'gtdtV_)A5^rYuev9'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.content)

train = pd.read_csv(get_object_storage_file_with_credentials_f2ced6cdb4894c42945223d461f16d59('notebooks', 'train.csv'), dtype={"Age": np.float64, "Fare": np.float64},)
test = pd.read_csv(get_object_storage_file_with_credentials_f2ced6cdb4894c42945223d461f16d59('notebooks', 'test.csv'), dtype={"Age": np.float64, "Fare": np.float64},)


In [4]:
# Preview of the training data
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [6]:
#Data audit
## Age has some missing values that can be imputed for downstream analysis; 
## Cabin has too many missings to be immediately useful
train.describe(include='all')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Graham, Mr. George Edward",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [7]:
#Replace missing values in Age
##Compute the average age for passengers by sex and class
train_aggSexPclass = train.groupby(['Sex','Pclass'], as_index=False).aggregate(np.mean).drop(['PassengerId','Survived','SibSp','Parch','Fare'], axis=1)
train_aggSexPclass = train_aggSexPclass.rename(columns={'Age': 'Age_Mean'})
print(train_aggSexPclass)


      Sex  Pclass   Age_Mean
0  female       1  34.611765
1  female       2  28.722973
2  female       3  21.750000
3    male       1  41.281386
4    male       2  30.740707
5    male       3  26.507589


In [8]:
# Merge the mean Age values by Sex and Pclass with the primary dataframe
train_merge = pd.merge(train,train_aggSexPclass,on=['Sex','Pclass'])
# Replace the missing values of Age with Age_Mean, then drop Age_Mean
train_merge.loc[train_merge.Age.isnull(), 'Age'] = train_merge.loc[train_merge.Age.isnull(), 'Age_Mean'] 
train_merge = train_merge.drop('Age_Mean', axis=1)
train_merge.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,891.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Graham, Mr. George Edward",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.318643,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,13.281103,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,21.750000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,26.507589,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,36.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [9]:
# Create an array of numeric fields that can be used to train the estimator
train_data = train_merge.drop(['PassengerId','Name','Sex','Ticket','Cabin','Embarked'], axis=1).values


In [10]:
# Training...
forest_estimator = RandomForestClassifier(n_estimators=10)
forest_model = forest_estimator.fit( train_data[0::,1::], train_data[0::,0] )


In [11]:
# Merge the mean Age values by Sex and Pclass with the primary test dataframe
test_merge = pd.merge(test,train_aggSexPclass,on=['Sex','Pclass'])
# Replace the missing values of Age with Age_Mean, then drop Age_Mean
test_merge.loc[test_merge.Age.isnull(), 'Age'] = test_merge.loc[test_merge.Age.isnull(), 'Age_Mean'] 
# Replace the missing values of Fare with the overall (test) mean of Fare
test_merge['Fare_Mean'] = np.mean(test_merge.Fare)
test_merge.loc[test_merge.Fare.isnull(), 'Fare'] = test_merge.loc[test_merge.Fare.isnull(), 'Fare_Mean'] 
test_merge = test_merge.drop(['Age_Mean','Fare_Mean'], axis=1)
test_merge.describe(include='all')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,418.000000,418.000000,418,418,418.000000,418.000000,418.000000,418,418.000000,91,418
unique,NaN,NaN,418,2,NaN,NaN,NaN,363,NaN,76,3
top,NaN,NaN,"Hee, Mr. Ling",male,NaN,NaN,NaN,PC 17608,NaN,B57 B59 B63 B66,S
freq,NaN,NaN,1,266,NaN,NaN,NaN,5,NaN,3,270
mean,1100.500000,2.265550,NaN,NaN,29.579566,0.447368,0.392344,NaN,35.627188,NaN,NaN
std,120.810458,0.841838,NaN,NaN,12.915662,0.896760,0.981429,NaN,55.840500,NaN,NaN
min,892.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,996.250000,1.000000,NaN,NaN,21.750000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,1100.500000,3.000000,NaN,NaN,26.507589,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,1204.750000,3.000000,NaN,NaN,36.000000,1.000000,0.000000,NaN,31.500000,NaN,NaN


In [12]:
# Create an array of numeric fields that can be used for scoring on the built model
test_data = test_merge.drop(['PassengerId','Name','Sex','Ticket','Cabin','Embarked'], axis=1).values


In [13]:
# Score test data ...
forest_scores = forest_model.predict(test_data).astype(int)


In [63]:
# ... and write out to file
predictions_file = open("titanic_python_01.csv", "wb")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["PassengerId","Survived"])
open_file_object.writerows(zip(test_merge['PassengerId'], forest_scores))
predictions_file.close()


Done.
